In [56]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time
import itertools
from collections import OrderedDict

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit
from fitter import special_functions as sf

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [57]:
reload(bs)
reload(dl)
reload(fit)
reload(sf)

<module 'fitter.special_functions' from '../fitter\special_functions.pyc'>

### Make fit

In [68]:
p_dict = {
    'order' : {
        'fit' : 'nnlo', # 'nlo', 'nnlo', or 'nnnlo'
        'vol' : 0, # max 10
        'exclude' : [] # put LECs here
    }, 
    'bs_N' : 1,  # if 0, use full list
    'fit_type' : 'xpt-taylor', #'ma', 'ma-taylor, 'xpt', or 'xpt-taylor'; also 'simultaneous', 'ma-old'
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'
    'chained' : True,
    
    'use_prior' : True,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400', u'a15m130', u'a15m220', u'a15m310', u'a15m350', 'a15m400'],
    
    'make_plots' : False,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,
}

In [71]:
fit_results = OrderedDict()

nnlo_lecs = ['A_a', 'A_k', 'A_p']
for k in range(len(nnlo_lecs)+1):
    for subset in itertools.combinations(nnlo_lecs, k):
        print list(subset)
        p_dict['order']['exclude'] = list(subset)
        
        included_lecs = list(set(nnlo_lecs).difference(subset))

        t0 = time.time()

        # Load data
        data_loader = dl.data_loader()
        fit_data = data_loader.get_fit_data()

        # Get prior
        prior = None
        if p_dict['use_prior']:
            prior = data_loader.get_prior(p_dict['fit_type'])

        # Make bootstrapper
        bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'], chain_fits=p_dict['chained'],
                                       fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])

        if p_dict['make_plots']:
            data_loader.save_plots(
                bootstrapper.make_plots(
                    show_error_ellipses=p_dict['show_error_ellipses'], 
                    show_bootstrap_histograms=p_dict['show_bs_histograms']),
                output_filename="fits_by_lec/"+bootstrapper.fit_type+'_'+bootstrapper.order['fit']+'_sans_'+str(p_dict['order']['exclude'])
            )

        # Save results for comparison
        fit_results[str(included_lecs)] = bootstrapper.get_fit_info()

        t1 = time.time()

        print "\nTotal time (s): ", t1 - t0, "\n"

[]
Making fits...
100% complete Time (s):  2.64899992943
Compiling results...

Total time (s):  2.95399999619 

['A_a']
Making fits...
100% complete Time (s):  2.62199997902
Compiling results...

Total time (s):  2.92499995232 

['A_k']
Making fits...
100% complete Time (s):  2.66199994087
Compiling results...

Total time (s):  2.96399998665 

['A_p']
Making fits...
100% complete Time (s):  2.62999987602
Compiling results...

Total time (s):  2.91699981689 

['A_a', 'A_k']
Making fits...
100% complete Time (s):  2.68299984932
Compiling results...

Total time (s):  2.97199988365 

['A_a', 'A_p']
Making fits...
100% complete Time (s):  2.61400008202
Compiling results...

Total time (s):  2.90000009537 

['A_k', 'A_p']
Making fits...
100% complete Time (s):  2.59200000763
Compiling results...

Total time (s):  2.88100004196 

['A_a', 'A_k', 'A_p']
Making fits...
100% complete Time (s):  2.62599992752
Compiling results...

Total time (s):  2.90400004387 



In [70]:
# These axes compare fits
ax_fits = plt.axes([0.10,0.10,0.49,0.8])

y=0
labels = np.array([])
for results in [fit_results]:
    #plt.axhline(y-0.5, ls='--')
    for name in results.keys():
        x = gv.mean(gv.gvar(results[name]['fit']))
        xerr = gv.sdev(gv.gvar(results[name]['fit']))
        plt.errorbar(x=x, y=y, xerr=xerr, yerr=0.0,
                     fmt = 'ro', capsize=0.0, mec='white', ms=25.0, alpha=0.6,
                     ecolor='b', elinewidth=25.0)
        y = y + 1
        labels = np.append(labels, str(name))
        
        # Add band for FLAG
        #if name == 'FLAG':
        #    plt.axvspan(x-xerr, x+xerr, alpha=0.3, color='g', label='FLAG')

ymax = y
            
# Show model average
#avg = model_average(fit_results, 'delta_su2')
#pm = lambda g, k : gv.mean(g) + k*gv.sdev(g)
#plt.axvspan(pm(avg, -1), pm(avg, +1), alpha=0.3, color='m', label='model avg')
#plt.axvline(pm(avg, 0), ls='-.', color='m')
        
plt.legend()

plt.yticks(range(len(labels)), labels, fontsize=15, rotation=65)
plt.ylim(-1, y)

#plt.title(particle+", "+abbr, fontsize=30)
plt.xlabel('$F^\pm_K/F^\pm_\pi$', fontsize=24)
plt.title('NNLO Fits,\n specified LECs only', fontsize=24)

plt.grid(ls='--')

# These axes compare the logGBFs
ax_logGBF = plt.axes([0.60,0.10,0.09,0.8])

# Get max logGBF
logGBF_max = np.nanmax([gv.mean(gv.gvar(fit_results[model]['logGBF'])) 
                       for model in fit_results.keys()])

y=0
labels = np.array([])
for results in [fit_results]:
    for name in results.keys():
        logGBF = gv.mean(gv.gvar(results[name]['logGBF']))
        x = np.exp(logGBF - logGBF_max)
        
        plt.axvline(x, ls='--', alpha=0.4)
        plt.scatter(x=x, y=y, color='r')
        y = y + 1
        labels = np.append(labels, str(name))

plt.yticks([])
#plt.xlim(-0.05, 1.05)
plt.ylim(-1, ymax)
plt.xlabel("Norm\nGBF", fontsize=24)

# These axes compare the Q-values
ax_Q = plt.axes([0.70,0.10,0.09,0.8])

y=0
labels = np.array([])
for results in [fit_results]:
    for name in results.keys():
        x = gv.mean(gv.gvar(results[name]['Q']))
        plt.axvline(x, ls='--', alpha=0.4)
        plt.scatter(x=x, y=y, color='r')
        y = y + 1
        labels = np.append(labels, str(name))

#for ti in np.arange(5)/4.0:
#    plt.axvline(ti, ls='--', alpha=0.2)

plt.yticks([])
plt.xlim(-0.05, 1.05)
plt.ylim(-1, ymax)
plt.xlabel("$Q$", fontsize=24)


# These axes compare the reduced chi2
ax_chi2 = plt.axes([0.80,0.10,0.1,0.8])

y=0
labels = np.array([])
for results in [fit_results]:
    for name in results.keys():
        x = gv.mean(gv.gvar(results[name]['chi2/df']))
        plt.axvline(x, ls='--', alpha=0.4)
        plt.scatter(x=x, y=y, color='r')
        y = y + 1
        labels = np.append(labels, str(name))

#for ti in np.arange(9)/4.0:
#    plt.axvline(ti, ls='--', alpha=0.2)

plt.yticks([])
plt.xlim(0.25, 1.75)
plt.ylim(-1, ymax)
plt.xlabel(r"$\chi^2_\nu$", fontsize=24)

fig = plt.gcf()
plt.close()

data_loader.save_plots(fig, output_filename='temp')

Done.
